In [1]:
def reproduceResult():
  seed_value= 0

  
  with tf.device("/cpu:0"):
    ...


  os.environ['PYTHONHASHSEED']=str(seed_value)
  np.random.seed(0)
  rn.seed(0)


  session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, 
                                          inter_op_parallelism_threads=1)


  tf.compat.v1.set_random_seed(seed_value)
  sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
  tf.compat.v1.keras.backend.set_session(sess)
  tf.compat.v1.keras.backend.clear_session()

  


In [2]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
  
import os 
import numpy as np
import tensorflow as tf
import random as rn
from tensorflow import keras

reproduceResult()
# %tensorflow_version 2.x
# import tensorflow as tf
# tf.test.gpu_device_name()
# from scipy import integrate
# import os
# import numpy as np
# from tensorflow import keras
import tempfile
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
# import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K


from tensorflow.keras import callbacks

from keras_lr_finder import LRFinder
from clr.clr_callback import CyclicLR

import tensorflow_model_optimization as tfmot



import keras_tuner
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from attention import Attention

C:\Users\moshi\AppData\Local\Temp/ipykernel_13036/628588012.py:43: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
df1 = pd.read_csv('qnli1.csv',index_col=False, encoding='iso-8859-1', 
                                warn_bad_lines=True, error_bad_lines=False)

df2 = pd.read_csv('qnli1.csv',index_col=False, encoding='iso-8859-1', 
                                warn_bad_lines=True, error_bad_lines=False)


C:\Users\moshi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\moshi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df1.dropna
df2.dropna

<bound method DataFrame.dropna of                                                question  \
0              When did the third Digimon series begin?   
1     Which missile batteries often have individual ...   
2     What two things does Popper argue Tarski's the...   
3     What is the name of the village 9 miles north ...   
4              What famous palace is located in London?   
...                                                 ...   
1427  The first American to travel into space was whom?   
1428  How many years does a president have in office...   
1429     What is the second principle of Protestantism?   
1430  WHat is the earliest surviving use of the clan...   
1431                                What are morphemes?   

                                               sentence           label  
0     Unlike the two seasons before it and most of t...  not_entailment  
1     When MANPADS is operated by specialists, batte...  not_entailment  
2     He bases this interpretation 

In [5]:
df1["label"].value_counts()

entailment        759
not_entailment    673
Name: label, dtype: int64

In [6]:
df2["label"].value_counts()

entailment        759
not_entailment    673
Name: label, dtype: int64

In [7]:
# taking acceptable and deleting unacceptable
df3 = df1[df1['label'] == 'entailment']

In [8]:
df3.head

<bound method NDFrame.head of                                                question  \
2     What two things does Popper argue Tarski's the...   
3     What is the name of the village 9 miles north ...   
5     When is the term 'German dialects' used in reg...   
6     What was the name of the island the English tr...   
8      What does the word 'customer' properly apply to?   
...                                                 ...   
1422     Where was the band Talking Heads based out of?   
1428  How many years does a president have in office...   
1429     What is the second principle of Protestantism?   
1430  WHat is the earliest surviving use of the clan...   
1431                                What are morphemes?   

                                               sentence       label  
2     He bases this interpretation on the fact that ...  entailment  
3     On 31 December 1853, the Ottoman forces at Cal...  entailment  
5     When talking about the German language, the t

In [9]:
# randomly picking 250 rows
df3 = df3.sample(n=250, replace=True)

In [10]:
df3["label"].value_counts()

entailment    250
Name: label, dtype: int64

In [11]:
# taking unacceptable and deleting unacceptable
df4 = df2[df2['label'] == 'not_entailment']

In [12]:
# randomly picking 250 rows
df4 = df4.sample(n=250, replace=True)

In [13]:
df4["label"].value_counts()

not_entailment    250
Name: label, dtype: int64

In [14]:
#concating acceptable and unacceptable
frames = [df3, df4]

temp = pd.concat(frames)

In [15]:
temp["label"].value_counts()

entailment        250
not_entailment    250
Name: label, dtype: int64

In [16]:
train, test = train_test_split(temp, test_size=0.2, stratify = temp['label'], random_state = 42)
num_classes = 2
embed_num_dims = 300
max_seq_len = 50

x_train = train['question'] + train['sentence']
x_test = test['question'] + test['sentence']

y_train = train['label']
y_test = test['label']

texts_train = x_train
texts_test = x_test

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train['question'] + train['sentence'])

sequence_train = tokenizer.texts_to_sequences(texts_train)
sequence_test = tokenizer.texts_to_sequences(texts_test)

index_of_words = tokenizer.word_index

vocab_size = len(index_of_words) + 1

print('Number of unique words: {}'.format(len(index_of_words)))

X_train_pad = pad_sequences(sequence_train, maxlen = max_seq_len, padding='pre' )
X_test_pad = pad_sequences(sequence_test, maxlen = max_seq_len,  padding='pre')

print(X_train_pad)


encoding = {
    'entailment': 1,
    'not_entailment': 0
}

y_train = [encoding[x] for x in train['label']]
y_test = [encoding[x] for x in test['label']]



y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

def create_embedding_matrix(word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    with open('C:/Users/moshi/Python Code/Untitled Folder/cc.en.300.vec',encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]
    return embedding_matrix


embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)
print(embedd_matrix.shape)


Number of unique words: 4183
[[ 267  268 1984 ...    1 2002  461]
 [   0    0    0 ...  673  463  674]
 [  15 1008 1009 ...    3 1028 1029]
 ...
 [   0    0    0 ...    4  999  349]
 [   0    0    0 ...    1  352 1977]
 [   0    0    0 ... 1000 1675 4183]]
(4184, 300)


# Random Search


In [17]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import  convert_variables_to_constants_v2_as_graph

def get_flops(model):
    concrete = tf.function(lambda inputs: model(inputs))
    concrete_func = concrete.get_concrete_function(
        [tf.TensorSpec([1, *inputs.shape[1:]]) for inputs in model.inputs])
    frozen_func, graph_def = convert_variables_to_constants_v2_as_graph(concrete_func)
    with tf.Graph().as_default() as graph:
        tf.graph_util.import_graph_def(graph_def, name='')
        run_meta = tf.compat.v1.RunMetadata()
        opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
        flops = tf.compat.v1.profiler.profile(graph=graph, run_meta=run_meta, cmd="op", options=opts)
        return flops.total_float_ops

In [18]:
from tensorflow.keras import regularizers
import time
LOG_DIR = f"{int(time.time())}"
seed_value= 0



def build_model(hp):
  
  reproduceResult()

  print('Ya it comes here')
  unit_attention = hp.Int("attention_unit",min_value =32, max_value = 128, step = 16)
  fake_val = hp.Int("cnn_1_unit",min_value =16, max_value = 96, step = 16)
  cnn_1_unit = hp.Int("cnn_1_unit",min_value =16, max_value = 96, step = 16)
  cnn_1_dropout = hp.Float("cnn_1_dropout",min_value = 0.1,max_value = 0.3,step = 0.1)

  lstm_unit = hp.Int("lstm_unit",min_value =64, max_value = 256, step = 32)
  lstm_dropout = hp.Float("lstm_dropout",min_value = 0.1,max_value = 0.5,step = 0.1)




  seq_input = keras.layers.Input(shape=(max_seq_len,))

  embedded = keras.layers.Embedding(vocab_size,
                          embed_num_dims,
                          input_length = max_seq_len,
                          weights = [embedd_matrix])(seq_input)

  cnn = keras.layers.Conv1D(cnn_1_unit,3,kernel_regularizer=regularizers.l2(1e-4),
                            bias_regularizer=regularizers.l2(1e-2),
                            activity_regularizer=regularizers.l2(1e-4))(embedded)
  cnn = keras.layers.Activation(activation='relu')(cnn)
  cnn = keras.layers.BatchNormalization()(cnn)
  cnn = keras.layers.Dropout(cnn_1_dropout,seed=seed_value)(cnn)


  lstm = keras.layers.Bidirectional(keras.layers.LSTM(lstm_unit, recurrent_regularizer=regularizers.l2(1e-4),
                                                      return_sequences=True,kernel_regularizer=regularizers.l2(1e-4),
                                                      bias_regularizer=regularizers.l2(1e-2),
                                                      activity_regularizer=regularizers.l2(1e-4),input_shape =(48,)))(cnn)
  lstm = keras.layers.Activation(activation='relu')(lstm)
  lstm = keras.layers.BatchNormalization()(lstm)
  lstm = keras.layers.Dropout(lstm_dropout,seed=seed_value)(lstm)
  
  

  max_pooling = keras.layers.GlobalMaxPooling1D()(lstm)
  output = keras.layers.Dense(num_classes, activation='softmax')(max_pooling)

  model = keras.Model(inputs = [seq_input], outputs = output)
  model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

  model.summary()
  
  print("The FLOPs is:{}".format(get_flops(model)) ,flush=True )
    
  print(" ")

  return model



clr_step_size = int((len(X_train_pad)/64))
base_lr = 1e-3
max_lr = 6e-3
mode = 'exp_range'


clr = CyclicLR(base_lr = base_lr, max_lr = max_lr, step_size = clr_step_size, mode = mode)

stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=5,
                              restore_best_weights=True,
                              verbose=0, mode='max')

tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 40,
    executions_per_trial = 1,
    directory = LOG_DIR
    )
  
tuner.search(x=X_train_pad,y = y_train,epochs = 25, batch_size = 4,callbacks = [stop,clr], 
             validation_data = (X_test_pad,y_test))


tuner.results_summary()

Trial 40 Complete [00h 00m 46s]
val_accuracy: 0.6800000071525574

Best val_accuracy So Far: 0.7099999785423279
Total elapsed time: 00h 40m 28s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in 1653506366\untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
attention_unit: 32
cnn_1_unit: 48
cnn_1_dropout: 0.2
lstm_unit: 160
lstm_dropout: 0.30000000000000004
Score: 0.7099999785423279
Trial summary
Hyperparameters:
attention_unit: 64
cnn_1_unit: 64
cnn_1_dropout: 0.2
lstm_unit: 128
lstm_dropout: 0.1
Score: 0.7099999785423279
Trial summary
Hyperparameters:
attention_unit: 112
cnn_1_unit: 48
cnn_1_dropout: 0.2
lstm_unit: 64
lstm_dropout: 0.1
Score: 0.699999988079071
Trial summary
Hyperparameters:
attention_unit: 80
cnn_1_unit: 96
cnn_1_dropout: 0.1
lstm_unit: 160
lstm_dropout: 0.2
Score: 0.699999988079071
Trial summary
Hyperparameters:
attention_unit: 64
cnn_1_unit: 96
cnn_1_dropout: 0.30000000000000004
lstm_un

In [19]:
tuner.search_space_summary()

Search space summary
Default search space size: 5
attention_unit (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
cnn_1_unit (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 96, 'step': 16, 'sampling': None}
cnn_1_dropout (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.3, 'step': 0.1, 'sampling': None}
lstm_unit (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 256, 'step': 32, 'sampling': None}
lstm_dropout (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': None}


In [ ]:
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
model = build_model(best_hps[0])

In [22]:
history = best_model.fit(x=X_train_pad,y = y_train,epochs = 20, batch_size = 4,callbacks = [stop,clr], 
             validation_data = (X_test_pad,y_test))

Epoch 1/20
100/100 [==============================] - 6s 39ms/step - loss: 0.3800 - accuracy: 0.9725 - val_loss: 1.3054 - val_accuracy: 0.6100
Epoch 2/20
100/100 [==============================] - 3s 32ms/step - loss: 0.5650 - accuracy: 0.9350 - val_loss: 2.3699 - val_accuracy: 0.6400
Epoch 3/20
100/100 [==============================] - 3s 33ms/step - loss: 0.4212 - accuracy: 0.9775 - val_loss: 1.4113 - val_accuracy: 0.6300
Epoch 4/20
100/100 [==============================] - 3s 32ms/step - loss: 0.3937 - accuracy: 0.9750 - val_loss: 1.4218 - val_accuracy: 0.6000
Epoch 5/20
100/100 [==============================] - 3s 32ms/step - loss: 0.3614 - accuracy: 0.9800 - val_loss: 1.3465 - val_accuracy: 0.6600
Epoch 6/20
100/100 [==============================] - 3s 32ms/step - loss: 0.3353 - accuracy: 0.9775 - val_loss: 1.4741 - val_accuracy: 0.6300
Epoch 7/20
100/100 [==============================] - 3s 32ms/step - loss: 0.3559 - accuracy: 0.9725 - val_loss: 1.9773 - val_accuracy: 0.6200

In [23]:
loss, accuracy = model.evaluate(X_train_pad, y_train, verbose=0)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test_pad, y_test, verbose=0)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9850
Testing Accuracy:  0.7100
